### Import libraries


In [91]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import numpy as np
import cv2


### Read the input data

In [92]:
def preprocess(file_name):
    image = cv2.imread(file_name)
    

In [93]:
with open('./data/driving_log.csv') as f:
    lines = f.readlines()

samples = []
for i, line in enumerate(lines[1:]):
    line = line.split(',')
    # file name
    file_name = './data/' + line[0].strip()
    stearing_angle = float(line[3])
    samples.append([file_name, stearing_angle, 0])

    # add flipped image
    samples.append([file_name, -stearing_angle, 1])



In [94]:
import sklearn


def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                file_name = batch_sample[0]
                image = cv2.imread(file_name)
                stearing_angle = batch_sample[1]
                flip_st = batch_sample[2]
                if flip_st == 1:
                    image = np.fliplr(image)
                # print(image[60:-20,:].shape)
                images.append(image[60:-20,:])
                angles.append(stearing_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)

            yield (X_train, y_train)
            

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


# Set our batch size
batch_size = 32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)


In [95]:
from tensorflow.keras.layers import Dense, Lambda, Input, Flatten
from tensorflow.keras.models import Sequential
# from tensorflow.keras.losses import MeanSquaredError


ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
# model.add(Input(shape=(row, col, ch)))
model.add(Lambda(lambda x: x/255.,
        input_shape=(row, col, ch),
        output_shape=(row, col, ch)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.summary()
model.fit(train_generator,
            steps_per_epoch=np.ceil(len(train_samples)/batch_size),
            validation_data=validation_generator,
            validation_steps=np.ceil(len(validation_samples)/batch_size),
            epochs=5, verbose=1)
            

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_35 (Lambda)           (None, 80, 320, 3)        0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 76800)             0         
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 76801     
Total params: 76,801
Trainable params: 76,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
402/402 [==============================] - 8s 21ms/step - loss: 57.3400 - val_loss: 0.0461
Epoch 2/5
402/402 [==============================] - 8s 21ms/step - loss: 0.0411 - val_loss: 0.0263
Epoch 3/5
402/402 [==============================] - 8s 20ms/step - loss: 0.0444 - val_loss: 0.1124
Epoch 4/5
402/402 [==============================] - 8s 20ms/step - loss: 0.05

### Define some variables to be used later

In [2]:

freeze_flag = True  # `True` to freeze layers, `False` for full training
weights_flag = 'imagenet' # 'imagenet' or None
preprocess_flag = True # Should be true for ImageNet pre-trained typically



### Load the pretrained model: 
    VGG16 (keras.applications.vgg16), 
    ResNet50 (keras.applications.resnet50), 
    Inception_V3 (keras.applications.inception_v3), ...
full list of models: https://keras.io/api/applications/

In [3]:
# Loads in InceptionV3
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions

from keras.preprocessing import image

# We can use smaller than the default 299x299x3 input for InceptionV3
# which will speed up training. Keras v2.0.9 supports down to 139x139x3
input_size = 299
model = InceptionV3(weights=weights_flag, include_top=True,
                        input_shape=(input_size, input_size, 3))





### Predict with the pretrained model

In [4]:
# Predict with loaded model
img_path = './images/elephant.jpg'
img = image.load_img(img_path, target_size=(input_size, input_size))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Perform inference on our pre-processed image
predictions = model.predict(x)

# Check the top 3 predictions of the model
print('Predicted:', decode_predictions(predictions, top=3)[0])

Predicted: [('n01871265', 'tusker', 0.64368755), ('n02504458', 'African_elephant', 0.17799999), ('n02504013', 'Indian_elephant', 0.082684584)]


### Load pretrained with dropped top layers

In order to change the input_size to 139x139x3, `include_top` must be set to `False`, which means the final fully-connected layer with 1,000 nodes for each ImageNet class is dropped, as well as a Global Average Pooling layer.

You can freeze layers by setting `layer.trainable` to False for a given `layer`. Within a `model`, you can get the list of layers with `model.layers`.

In [5]:
# We can use smaller than the default 299x299x3 input for InceptionV3
# which will speed up training. Keras v2.0.9 supports down to 139x139x3
# In order to do so, we also must set include_top to False, which means the final fully-connected layer with 1,000 nodes for each ImageNet class is dropped, as well as a Global Average Pooling layer.

input_size = 139
model = InceptionV3(weights=weights_flag, include_top=False,
                        input_shape=(input_size, input_size, 3))

if freeze_flag == True:
    for l in model.layers:
        l.trainable = False
    
model.summary()


Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 139, 139, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 69, 69, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 69, 69, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 69, 69, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

### Dropping layers
You can drop layers from a model with `model.layers.pop()`.

### Attaching new input layer with Lambda layer to the pretrained model

In [6]:
from keras.layers import Input, Lambda
import tensorflow as tf

# Makes the input placeholder layer 32x32x3 for CIFAR-10
cifar_input = Input(shape=(32,32,3))

# Re-sizes the input with Kera's Lambda layer & attach to cifar_input
resized_input = Lambda(lambda image: tf.image.resize( 
    image, (input_size, input_size)))(cifar_input)

# Feeds the re-sized input into Inception model
# You will need to update the model name if you changed it earlier!
inp = model(resized_input)

### Define Magic Layer

In [7]:
import tensorflow as tf
from tensorflow.python.ops import random_ops


class SMagicLayer(tf.keras.layers.Layer):
    def __init__(self,
                 filters=1,
                 has_bias=True,
                 activation=None,
                 seed=None,
                 **kwargs):
        super(SMagicLayer, self).__init__(**kwargs)
        self.has_bias = has_bias
        self.seed = seed
        self.activation = activation
        self.filters = filters

    def build(self, input_shape):
        print('input_shape', input_shape)
        w = input_shape[1]  # assuming input_shape=(None, w, h, c)
        h = input_shape[2]
        c = input_shape[3]
        # self.kernels = []
        # for _ in range(self.filters):
        #     self.kernels.append(tf.Variable(trainable=True, name=self.name + '_kernel',
        #                                     initial_value=random_ops.truncated_normal((w, h, c),
        #                                                                               mean=0.0,
        #                                                                               stddev=1.,
        #                                                                               dtype=tf.float32,
        #                                                                               seed=self.seed)))
        self.kernel = tf.Variable(trainable=True, name=self.name + '_kernel',
                                  initial_value=random_ops.truncated_normal((self.filters, w, h, c),
                                                                            mean=0.0,
                                                                            stddev=1.,
                                                                            dtype=tf.float32,
                                                                            seed=self.seed))

        if self.has_bias:
            self.bias = tf.Variable(trainable=True, name=self.name + '_bias',
                                    initial_value=random_ops.truncated_normal((self.filters, w, h),
                                                                              mean=0.0,
                                                                              stddev=1.,
                                                                              dtype=tf.float32,
                                                                              seed=self.seed))
        self.out_shape = (w, h, 1)

    def call(self, inputs):
        # print('inputs.shape', inputs.shape)
        # print('self.kernel.shape', self.kernel.shape)
        outputs = []
        for f in range(self.filters):
            o = tf.multiply(self.kernel[f, :, :], inputs)
            o = tf.math.reduce_sum(o, axis=3)
            if self.has_bias:
                o = o + self.bias[f, :]
            if self.activation is not None:
                o = self.activation(o)
            outputs.append(o)
        # print('outputs.shape', outputs.shape)
        # print('self.bias.shape', self.bias.shape)

        return tf.stack(outputs, axis=3)



### Define ExpSq Activation 

In [8]:
class ExpSqActivation(tf.keras.layers.Layer):
    def __init__(self, center=1.0, tau=0.45, bias=0.0):
        super().__init__()
        self.tau = tau
        self.bias = bias
        self.center = center

    def call(self, inputs):
        y = tf.exp(-1.0 * tf.square(self.center -
                                    inputs) / (2.0 * self.tau ** 2))
        if self.bias != 0:
            y = (1-self.bias) * y + self.bias
        return y


### Connect your top layers to the predefined model + customized input layer

In [21]:
# Imports fully-connected "Dense" layers & Global Average Pooling
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Conv2D, Softmax

# out = GlobalAveragePooling2D()(inp)
# out = Dense(400, activation='relu')(out)
# predictions = Dense(10, activation='softmax', name='predictions')(out)

# out = SMagicLayer(filters=10, activation=tf.keras.activations.relu)(inp)

out = SMagicLayer(filters=1)(inp)
# out = SMagicLayer(filters=10)(out)
# out = SMagicLayer(filters=1)(out)
# out = SMagicLayer(filters=1)(out)

# my architecture
# add conv full size here with 10 filters to generate 10 numbers
# these numbers are kind of like pattern matching for each class
# flatten()
# softmax()
# endof my architecture

out = Flatten()(out)
# out = Conv2D(filters=10, kernel_size=(3, 3))(out)
#out = Flatten()(inp)
out = Dense(10)(out)
# out = Softmax()(out)
out = Dense(10, activation='softmax')(out)


input_shape (None, 3, 3, 2048)


### Compile the new model

In [22]:
# Imports the Model API
from keras.models import Model

# Creates the model, assuming your final layer is named "predictions"
new_model = Model(inputs=cifar_input, outputs=out)

# Compile the model
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Check the summary of this new model to confirm the architecture
new_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
lambda (Lambda)              (None, 139, 139, 3)       0         
_________________________________________________________________
inception_v3 (Functional)    (None, 3, 3, 2048)        21802784  
_________________________________________________________________
s_magic_layer_3 (SMagicLayer (None, 3, 3, 1)           18441     
_________________________________________________________________
flatten_5 (Flatten)          (None, 9)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                100       
_________________________________________________________________
dense_10 (Dense)             (None, 10)                110 

### Keras Callbacks

There's two key callbacks to mention here, `ModelCheckpoint` and `EarlyStopping`. As the names may suggest, model checkpoint saves down the best model so far based on a given metric, while early stopping will end training before the specified number of epochs if the chosen metric no longer improves after a given amount of time.

To set these callbacks, you could do the following:
```
checkpoint = ModelCheckpoint(filepath=save_path, monitor='val_loss', save_best_only=True)
```
This would save a model to a specified `save_path`, based on validation loss, and only save down the best models. If you set `save_best_only` to `False`, every single epoch will save down another version of the model.
```
stopper = EarlyStopping(monitor='val_acc', min_delta=0.0003, patience=5)
```
This will monitor validation accuracy, and if it has not decreased by more than 0.0003 from the previous best validation accuracy for 5 epochs, training will end early.


You still need to actually feed these callbacks into `fit()` when you train the model (along with all other relevant data to feed into `fit`):
```
model.fit(callbacks=[checkpoint, stopper])
```

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
save_path = './checkpoints'
checkpoint = ModelCheckpoint(filepath=save_path, monitor='val_loss', save_best_only=True)
stopper = EarlyStopping(monitor='val_acc', min_delta=0.0003, patience=5)


In [22]:
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from keras.datasets import cifar10

(X_train, y_train), (X_val, y_val) = cifar10.load_data()

# One-hot encode the labels
label_binarizer = LabelBinarizer()
y_one_hot_train = label_binarizer.fit_transform(y_train)
y_one_hot_val = label_binarizer.fit_transform(y_val)

# Shuffle the training & test data
X_train, y_one_hot_train = shuffle(X_train, y_one_hot_train)
X_val, y_one_hot_val = shuffle(X_val, y_one_hot_val)

# We are only going to use the first 10,000 images for speed reasons
# And only the first 2,000 images from the test set
X_train = X_train[:10000]
y_one_hot_train = y_one_hot_train[:10000]
X_val = X_val[:2000]
y_one_hot_val = y_one_hot_val[:2000]

You can check out Keras's [ImageDataGenerator documentation](https://faroit.github.io/keras-docs/2.0.9/preprocessing/image/) for more information on the below - you can also add additional image augmentation through this function, although we are skipping that step here so you can potentially explore it in the upcoming project.

In [23]:
# Use a generator to pre-process our images for ImageNet
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

if preprocess_flag == True:
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
else:
    datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()

In [24]:
# Train the model
batch_size = 32
epochs = 5
# Note: we aren't using callbacks here since we only are using 5 epochs to conserve GPU time
new_model.fit(datagen.flow(X_train, y_one_hot_train, batch_size=batch_size), 
                    steps_per_epoch=len(X_train)/batch_size, epochs=epochs, verbose=1, 
                    validation_data=val_datagen.flow(X_val, y_one_hot_val, batch_size=batch_size),
                    validation_steps=len(X_val)/batch_size)
#                    callbacks=[checkpoint, stopper])

Epoch 1/5
312/312 [==============================] - 12s 30ms/step - loss: 32.5096 - accuracy: 0.2960 - val_loss: 8.5364 - val_accuracy: 0.5720
Epoch 2/5
312/312 [==============================] - 9s 27ms/step - loss: 6.1511 - accuracy: 0.6069 - val_loss: 4.0317 - val_accuracy: 0.6330
Epoch 3/5
312/312 [==============================] - 8s 27ms/step - loss: 2.8178 - accuracy: 0.6515 - val_loss: 2.4009 - val_accuracy: 0.6395
Epoch 4/5
312/312 [==============================] - 8s 27ms/step - loss: 1.6413 - accuracy: 0.6729 - val_loss: 1.6042 - val_accuracy: 0.6495
Epoch 5/5
312/312 [==============================] - 9s 27ms/step - loss: 1.2574 - accuracy: 0.6776 - val_loss: 1.5196 - val_accuracy: 0.6300


As you may have noticed, CIFAR-10 is a fairly tough dataset. However, given that we are only training on a small subset of the data, only training for five epochs, and not using any image augmentation, the results are still fairly impressive!

We achieved ~70% validation accuracy here, although your results may vary.